# Official Competition Protocol

In [2]:
from __future__ import division
from BenchExperiment import *
from competition_functions import *
import pandas
import pickle

## Setup competition objects

### Setup competition strains


#### Load reference growth rate data

In [3]:
growthRatesData = pandas.read_csv('/Users/ryan/Projects/infoload/2018-Fall_rpobBCFP_8hrPrelimCompetitions/CalcSelectionCoefficients/median_growth_rates_rpobBCFP8hrCompetitions.txt', sep='\t', usecols=['strain', 'median_r_val'])

In [4]:
strainNames = growthRatesData['strain'].unique().tolist()
strainNames

['M1', 'M10', 'M11', 'M12', 'M2', 'M3', 'M4', 'M5', 'M6', 'M8', 'M9', 'WT']

In [5]:
growthRatesData

,strain,median_r_val
0,M1,1.376634
1,M10,1.092049
2,M11,1.389472
3,M12,1.205116
4,M2,1.314519
5,M3,1.123365
6,M4,0.675563
7,M5,1.218071
8,M6,1.128701
9,M8,1.082592


In [6]:
growthRates = {}
for strainName in strainNames:
    growthRates[strainName] = growthRatesData.query("strain==\'"+strainName+"\'")['median_r_val'].item()
growthRates

{'M1': 1.37663423857,
 'M10': 1.0920485097600001,
 'M11': 1.38947171356,
 'M12': 1.20511595666,
 'M2': 1.31451914198,
 'M3': 1.12336512666,
 'M4': 0.675563095451,
 'M5': 1.2180707637200001,
 'M6': 1.12870130274,
 'M8': 1.08259235934,
 'M9': 0.8656132403229999,
 'WT': 1.32325537071}

#### Instantiate Strain objects

In [7]:
strains = {}
gfpMarkedStrain = 'WT'
for strainName in strainNames:
    strains[strainName] = Strain(name=strainName, 
                                    max_growth_rate=growthRates[strainName], 
                                    mean_lag_exit_time=1.5,
                                    stdev_lag_exit_time=0.1,
                                    marker=('gfp' if strainName==gfpMarkedStrain else 'rfp'))

In [8]:
freezedowns = {}
for strainName, strain in strains.items():
    freezedowns[strainName] = CultureVolume(media=[Media(volume=1.0, carrying_capacity=0.0)], inoculums=[Inoculum(strain=strain, cell_count=1.0e9)], name="Freezedown "+strainName)

### Setup competition media

In [9]:
LBStock = CultureVolume(media=Media(volume=6000, carrying_capacity=numpy.random.normal(3.0e9, 1.0e9)), name="LB Stock")

tetStockVolume = 10.0 #mL
tetStockConc   = 15.0 #mg/mL
tetStock= CultureVolume(media=Media(volume=tetStockVolume, carrying_capacity=0), drugs=[Drug(name='Tet', mg=tetStockConc*tetStockVolume)], name="Tet Stock")

LBtetMedia = CultureVolume(name="LB-tet Media Stock")
LBtetMedia.add(LBStock.sample(5000.0))
LBtetMedia.add(tetStock.sample(5.0))
LBtetMedia.info()

CultureVolume LB-tet Media Stock:
--------------------------------
	Volume			= 4847.342 mL
	Carrying Capacity	= 2.07E+09 cfu/mL
	Density			= 0.00E+00 cfu/mL
	Total Cell Count	= 0.00E+00 cfu
	(no inoculums)
	Drug Tet:
		concentration	= 0.01564 mg/mL


### Setup competition equipment

In [10]:
incubator = Incubator(set_temp=37.0, temp_std_batch=0.5, temp_std_location=0.25, temp_std_transient=0.01)
flow      = FlowCytometer()

### Setup competition data collection data structures

In [11]:
allCompetitionData_alleleFreqs = pandas.DataFrame(columns=['competition', 'competition_rep', 'time', 'sample_rep', 'allele', 'allele_freq'])
allCompetitionData_popSizes    = pandas.DataFrame(columns=['competition', 'competition_rep', 'time', 'sample_rep', 'population', 'pop_size'])

allCompetitionData_growthCurves = pandas.DataFrame(columns=['culture', 'competition_rep', 'time', 'temp', 'density']) 

## Start overnight cultures
* Pipette 200 mL of LB media into the overnight media prep reservoir.
* Pipette 5 mL of LB only media into the overnight neg ctrl tube
* Pipette 195 uL of Tetracycline stock into the overnight media prep reservoir.
* Pipette 5.0 mL of LB+Tet media into the 1X overnight culture tube for each strain
* Pipette 4.95 mL of LB+Tet media into the 0.1X, 0.01X, 0.001X overnight tubes for each strain

For each of the the competition strains:
    * Insert freezedown culture scraping into the corresponding 1X overnight culture tubes
    * Pipette 50 uL from the 1X overnight tube to the 0.1X overnight tube
    * Pipette 50 uL from the 0.1X overnight tube to the 0.01X overnight tube
    * Pipette 50 uL from the 0.01X overnight tube to the 0.001X overnight tube
    * Place all overnight culture tubes in shaking incubator

In [12]:
def inoculateOvernightCulture(strain, media, media_volume=5.0, name=""):
    overnight = CultureVolume(name="Overnight "+name)
    overnight.add(media.sample(media_volume))
    overnight.add(freezedowns[strain.name].sample(0.010))
    return overnight

In [13]:
LBStock = CultureVolume(media=Media(volume=1000, carrying_capacity=numpy.random.normal(3.0e9, 1.0e9)), name="LB Stock")
tetStock= CultureVolume(media=Media(volume=1.0, carrying_capacity=0), drugs=[Drug(name='Tet', mg=15.0)], name="Tet Stock")

In [14]:
overnights = {}

overnightMediaPrepReservoir = CultureVolume(name="Overnight Media Prep Reservoir")
overnightMediaPrepReservoir.add(LBStock.sample(150.0))
overnightMediaPrepReservoir.info()

# overnights['NegCtrl'] = CultureVolume(name="Overnight (-)")
# overnights['NegCtrl'].add(overnightMediaPrepReservoir.sample(5.0))
# overnights['NegCtrl'].info()

overnightMediaPrepReservoir.add(tetStock.sample(0.145))
overnightMediaPrepReservoir.info()

numOvernightsPerStrain = 3
for strainName, strain in strains.items():
    overnights[strainName] = []
    for overnightRep in range(numOvernightsPerStrain):
        overnightName = strainName+"r"+str(overnightRep)
        overnights[strainName].append( inoculateOvernightCulture(strain=strain, 
                                                                 media=overnightMediaPrepReservoir, 
                                                                 media_volume=5.0, 
                                                                 name=overnightName) )
        
incubator.incubate(cultures=numpy.concatenate(overnights.values()), time=24, dt=0.01)

for overnight in numpy.concatenate(overnights.values()):
    overnight.info()
        

CultureVolume Overnight Media Prep Reservoir:
--------------------------------
	Volume			= 151.338 mL
	Carrying Capacity	= 3.18E+09 cfu/mL
	Density			= 0.00E+00 cfu/mL
	Total Cell Count	= 0.00E+00 cfu
	(no inoculums)
	(no drugs)
CultureVolume Overnight Media Prep Reservoir:
--------------------------------
	Volume			= 151.482 mL
	Carrying Capacity	= 3.18E+09 cfu/mL
	Density			= 0.00E+00 cfu/mL
	Total Cell Count	= 0.00E+00 cfu
	(no inoculums)
	Drug Tet:
		concentration	= 0.01424 mg/mL
CultureVolume Overnight M11r0:
--------------------------------
	Volume			= 5.121 mL
	Carrying Capacity	= 3.17E+09 cfu/mL
	Density			= 3.17E+09 cfu/mL
	Total Cell Count	= 1.63E+10 cfu
	Inoculum M11:
		density		= 3.17E+09 cfu/mL
		cell count	= 1.63E+10 cfu
	Drug Tet:
		concentration	= 0.01418 mg/mL
CultureVolume Overnight M11r1:
--------------------------------
	Volume			= 4.875 mL
	Carrying Capacity	= 3.17E+09 cfu/mL
	Density			= 3.17E+09 cfu/mL
	Total Cell Count	= 1.55E+10 cfu
	Inoculum M11:
		density		= 

## Dilute back each overnight culture:
#### Prep diluted-back culture flasks with media:
* For each of the following diluted-back cultures:
    * rpobWT_bc11GFP (3 culture tubes)
    * rpobM3_bc2RFP (3 culture tubes)
    * rpobM4_bc8RFP (3 culture tubes)
    * rpobM5_bc9RFP (3 culture tubes)
    * rpobM6_bc10RFP (3 culture tubes)
    * rpobM8_bc12RFP (3 culture tubes)
    * rpobM9_bc3RFP (3 culture tubes)
    * rpobM10_bc13RFP (3 culture tubes)
    * neg ctrl (1 culture tubes)
* Pipette 25 mL of LB media into each diluted-back culture flask.
* Pipette 5 mL of LB only media into the diluted-back neg ctrl tube.
* Pipette 25 uL of Tetracycline media into each diluted-back culture flask.

#### Dilute back each overnight culture:
* Pipette 250 uL of each overnight culture into the corresponding diluted-back culture flask(s).
* Place the diluted-back culture flasks in shaking incubator.
* Note starting time of diluted-back culture growth. Prepare to take diluted-back cultures out of incubator exactly 1,5 hours later for immediate use.

In [15]:
def inoculateDilutedBackCulture(strain, media, overnight, media_volume=25.0, overnight_volume=250.0, name=""):
    dilutedBack = CultureVolume(name="Diluted-back "+name)
    dilutedBack.add(media.sample(media_volume))
    dilutedBack.add(tetStock.sample(media_volume/1000))
    dilutedBack.add(overnight.sample(overnight_volume))
    return dilutedBack

In [16]:
dilutedBacks = {}

dilutedBackMediaVolume     = 25.0
dilutedBackOvernightVolume = 0.250
numDilutedBacksPerStrain   = 1
for strainName, strain in strains.items():
    dilutedBackName = strainName
    overnight = [on for on in overnights[strainName] if on.totalVolume()>dilutedBackOvernightVolume][0]
    dilutedBacks[strainName] = inoculateDilutedBackCulture(strain, media=LBStock, overnight=overnight, 
                                                            media_volume=dilutedBackMediaVolume, 
                                                            overnight_volume=dilutedBackOvernightVolume, 
                                                            name=dilutedBackName)
              
# Not currently explicitly simulated lag phase exit. 
# Diluted-back cultures are incubated for 1.5hrs in the real protocol. 
# Here, simulating 0.75hrs of incubated growth, implying assumed lag phase duration of 0.75hrs.
incubator.incubate(cultures=dilutedBacks.values(), time=0.8, dt=0.01)

for dilutedBack in dilutedBacks.values():
    dilutedBack.info()   

CultureVolume Diluted-back M11:
--------------------------------
	Volume			= 24.198 mL
	Carrying Capacity	= 3.18E+09 cfu/mL
	Density			= 3.22E+07 cfu/mL
	Total Cell Count	= 7.80E+08 cfu
	Inoculum M11:
		density		= 3.22E+07 cfu/mL
		cell count	= 7.80E+08 cfu
	Drug Tet:
		concentration	= 0.01578 mg/mL
CultureVolume Diluted-back M6:
--------------------------------
	Volume			= 24.771 mL
	Carrying Capacity	= 3.18E+09 cfu/mL
	Density			= 3.36E+07 cfu/mL
	Total Cell Count	= 8.33E+08 cfu
	Inoculum M6:
		density		= 3.36E+07 cfu/mL
		cell count	= 8.33E+08 cfu
	Drug Tet:
		concentration	= 0.01501 mg/mL
CultureVolume Diluted-back M10:
--------------------------------
	Volume			= 25.133 mL
	Carrying Capacity	= 3.18E+09 cfu/mL
	Density			= 3.26E+07 cfu/mL
	Total Cell Count	= 8.18E+08 cfu
	Inoculum M10:
		density		= 3.26E+07 cfu/mL
		cell count	= 8.18E+08 cfu
	Drug Tet:
		concentration	= 0.01451 mg/mL
CultureVolume Diluted-back M5:
--------------------------------
	Volume			= 26.164 mL
	Carrying Cap

## Run Competitions

### Competition Parameters 

In [17]:
competitionInitStrainFreqs = { 
                                '2a1': {'WT':0.5, 'M3':0.5},
                                '2b1': {'WT':0.5, 'M10':0.5},
                                '2c1': {'WT':0.5, 'M4':0.5},
                                '2a2': {'WT':0.25, 'M3':0.75},
                                '2b2': {'WT':0.25, 'M10':0.75},
                                '2c2': {'WT':0.25, 'M4':0.75},
                                '2a3': {'WT':0.125, 'M3':0.875},
                                '2b3': {'WT':0.125, 'M10':0.875},
                                '2c3': {'WT':0.125, 'M4':0.875},
                                '4a1': {'WT':0.25, 'M3':0.25, 'M6':0.25, 'M8':0.25},
                                '4b1': {'WT':0.25, 'M4':0.25, 'M9':0.25, 'M10':0.25},
                                '4a2': {'WT':0.125, 'M3':(7/8)/3, 'M6':(7/8)/3, 'M8':(7/8)/3},
                                '4b2': {'WT':0.125, 'M4':(7/8)/3, 'M9':(7/8)/3, 'M10':(7/8)/3},
                                '8a1': {'WT':0.125, 'M3':0.25, 'M6':0.25, 'M8':0.25, 'M4':0.25, 'M9':0.25, 'M10':0.25}
                             }

In [18]:
competitionFlaskMediaVolume    = 25.0
competitionFlaskTransferVolume = 0.250

In [19]:
totalTime            = 24
transferTimeInterval = 4

### Initiate Competitions

#### Inoculate competition flasks
For each competition:
* Inoculate 0h Conical Tube with Players at Desired Initial Frequencies:
    * Pipette the <appropriate volume> (see spreadsheet) of each player’s diluted-back culture into the corresponding 0h conical tube.
* Transfer Culture from 0h Conical Tube to 4h Competition Flask:
    * Pipette 250 uL of the culture from the 0h conical tube to the 4h competition flask.
* Sample Culture for Flow Cytometry:
    * Pipette 1 mL of culture from the 4h competition flask into the corresponding 0h flow sample epi. 
* Incubate Competition Flasks:
    * Place 4h competition flask in 37C shaking incubator
* Sample Culture for Sequencing:
    * []Leave the remaining 0h conical tube contents (~24 mL) in what is now the 0h sequencing sample conical tube.

In [20]:
# LBStock = CultureVolume(media=Media(volume=1000, carrying_capacity=numpy.random.normal(3.0e9, 1.0e9)), name="LB Stock")
# tetStock= CultureVolume(media=Media(volume=1.0, carrying_capacity=0), drugs=[Drug(name='Tet', mg=15.0)], name="Tet Stock")

In [21]:
competitionFlasks = {}

In [22]:
for competition, initStrainFreqs in competitionInitStrainFreqs.items():
    competitionFlasks[competition] = CultureVolume(name="Competition "+competition+" 0h")
    for strainName, initFreq in initStrainFreqs.items():
        competitionFlasks[competition].add(dilutedBacks[strainName].sample(initFreq*competitionFlaskMediaVolume))

### Execute competition transfers and data collection sampling

#### Fill the Competition Flasks for the next transfer:
* Pipette 25 mL of LB media into the competition flask.
* Pipette 25 uL of Tetracycline into the competition flask.

#### Transfer Competition Culture to next Competition Flask:
* Pipette 250 uL of the culture from the 4h competition flask to the 8h competition flask.

#### Incubate Competition Flasks:
* Place 8h competition flask in 37C shaking incubator

#### Sample Culture for Flow Cytometry:
* Pipette 1 mL of culture from the 4h competition flask into the corresponding 4h flow sample epi. 

#### Sample Culture for Sequencing:
* Pour the remaining culture contents (~24 mL) of the 4h competition flask into the corresponding 4h sequencing sample conical tube.

In [23]:
def competitionFlaskTransfer(competition_flask, media, media_volume, transfer_volume, name=""):
    competitionFlask_next = CultureVolume(name="Competition "+name)
    competitionFlask_next.add(media.sample(competitionFlaskMediaVolume))
    competitionFlask_next.add(tetStock.sample(competitionFlaskMediaVolume/1000))
    competitionFlask_next.add(competition_flask.sample(transfer_volume))
    return competitionFlask_next

In [24]:
def collectEnumerationSampleData(competition_culture, sample_reps, t, data):
    for r in range(sample_reps):
        enumerationSample         = competition_culture.sample(1.0)
        enumerationDilutionSeries = DilutionSeries(enumerationSample)
        
        flowReadVolume    = 0.100
        flowReadDilutions = [-2, -1]
        for df in flowReadDilutions:
            flowEvents = flow.read(sample=enumerationDilutionSeries.getDilution(df), read_volume=flowReadVolume)
            for markerName, numEvents in flowEvents.iteritems():
                markerDensityEstimate = numEvents*(1.0/flowReadVolume) * 10**(numpy.abs(df))
                data['pop_sizes'].append( {'time': t, 'sample_rep': r+1, 
                                           'population': markerName, 'pop_size': markerDensityEstimate} )        

In [25]:
def collectSequencingSampleData(competition_culture, sample_reps, t, data):
    for r in range(sample_reps):
        sequencingSample         = competition_culture.sample( competition_culture.totalVolume() )
        
        # In lieu of simulating sequencing more directly, return estimated frequencies based on cell counts with small noise:
        trueCellCounts = sequencingSample.getCellCounts()
        print trueCellCounts
        estimatedCellCounts = {}
        for inoculumName, cellCount in trueCellCounts.iteritems():
            estimatedCellCounts[inoculumName] = int(numpy.random.normal(cellCount, cellCount*0.01))
        estimatedTotalCellCount = numpy.sum(estimatedCellCounts.values())
        print estimatedTotalCellCount

        for inoculumName, cellCount in estimatedCellCounts.iteritems():
            frequencyEstimate = cellCount/estimatedTotalCellCount
            data['allele_freqs'].append( {'time': t, 'sample_rep': r+1, 
                                          'allele': inoculumName, 'allele_freq': frequencyEstimate} )        
            

In [26]:
t = 0
while t < totalTime:
    for competition, competitionFlask in competitionFlasks.items():
        
        # Transfer Competition Culture to next Competition Flask:
        competitionFlask_next = competitionFlaskTransfer(competitionFlask, 
                                                     media=LBStock, media_volume=competitionFlaskMediaVolume, 
                                                     transfer_volume=competitionFlaskTransferVolume, 
                                                     name="Competition "+competition+" "+str(t+transferTimeInterval)+"h")

        competitionTimepointData = {'allele_freqs':[], 'pop_sizes':[]}
        
        #~~~~~~~~~~~~~~~~~~~~
        # Sample Competition Culture for Population Size Estimation (via flow cytometry enumeration):
        if(t==0):
            collectEnumerationSampleData(competition_culture=competitionFlask_next, 
                                         sample_reps=3, 
                                         t=t, 
                                         data=competitionTimepointData)
        else:
            collectEnumerationSampleData(competition_culture=competitionFlask, 
                                         sample_reps=3, 
                                         t=t, 
                                         data=competitionTimepointData)
        # Annotate this time point data with the competition name and rep number:
        competitionTimepointData['pop_sizes']    = [dict(d, competition=competition, competition_rep=-1) for d in competitionTimepointData['pop_sizes']]
        # Add this competition's data recorded for this time point to the overall dataset:
        allCompetitionData_popSizes    = allCompetitionData_popSizes.append(competitionTimepointData['pop_sizes'])
        
        #~~~~~~~~~~~~~~~~~~~~
        # Sample Competition for Allele Frequency Estimation (via sequencing):
        collectSequencingSampleData(competition_culture=competitionFlask, 
                                    sample_reps=1, 
                                    t=t, 
                                    data=competitionTimepointData)
        # Annotate this time point data with the competition name and rep number:
        competitionTimepointData['allele_freqs'] = [dict(d, competition=competition, competition_rep=-1) for d in competitionTimepointData['allele_freqs']]
        # Add this competition's data recorded for this time point to the overall dataset:
        allCompetitionData_alleleFreqs = allCompetitionData_alleleFreqs.append(competitionTimepointData['allele_freqs'])

        


#         exit()
        break
    break
        
        # Incubate the new  Competition Flasks:
#         competitionFlasks[competition] = competitionFlask_next
#         incubator.incubate(cultures=competitionFlasks.values(), time=transferTimeInterval, dt=0.01)

{'WT': 195983157, 'M6': 208530218, 'M8': 196875538, 'M3': 199827408}
805629977


In [27]:
print allCompetitionData_alleleFreqs

  competition competition_rep time sample_rep allele  allele_freq
0         4a1              -1    0          1     WT     0.241937
1         4a1              -1    0          1     M6     0.261845
2         4a1              -1    0          1     M8     0.244908
3         4a1              -1    0          1     M3     0.251309


In [28]:
print allCompetitionData_popSizes

   competition competition_rep time sample_rep population  pop_size
0          4a1              -1    0          1        gfp  186000.0
1          4a1              -1    0          1        rfp  660000.0
2          4a1              -1    0          1        gfp  203600.0
3          4a1              -1    0          1        rfp  623900.0
4          4a1              -1    0          1    gfp-rfp    1000.0
5          4a1              -1    0          2        gfp  178000.0
6          4a1              -1    0          2        rfp  588000.0
7          4a1              -1    0          2        gfp  202600.0
8          4a1              -1    0          2        rfp  624200.0
9          4a1              -1    0          2    gfp-rfp    1300.0
10         4a1              -1    0          3        gfp  180000.0
11         4a1              -1    0          3        rfp  638000.0
12         4a1              -1    0          3        gfp  193400.0
13         4a1              -1    0          3  